In [5]:
import pandas as pd
from tqdm import tqdm
from tools.utils import read_jsonl,get_type_from_label,get_prompt,post_process,process_label
from tools.inference import load_model_and_tokenizer,transformers_inference

DATA_SELECT = 'dev'
GLM3_MODEL_PATH = '../../checkpoint-5000'

- 前处理

In [2]:
# 数据加载 模型1：GLM3-FT加载
df = read_jsonl(f'data/{DATA_SELECT}.json')
stock_names = pd.read_excel('data/data-0520/标准名.xlsx',sheet_name='股票标准名')['标准股票名称'].to_list()
fund_names = pd.read_excel('data/data-0520/标准名.xlsx',sheet_name='基金标准名')['标准基金名称'].to_list()
glm3_finetune,tokenizer = load_model_and_tokenizer(GLM3_MODEL_PATH)

df['glm3_output'] = ''
df['type_'] = 0
df['prompt'] = ''
df['prompt2'] = ''
for idx,row in tqdm(df.iterrows(),total=df.shape[0]):
    input = row['input']
    output = transformers_inference(input,glm3_finetune,tokenizer)
    df.at[idx,'glm3_output'] = output
    df.at[idx,'type_'] = get_type_from_label(output)
    df.at[idx,'glm3_output'] = process_label(df.at[idx,'glm3_output']) # 个别原始小错处理
    prompts = get_prompt(row,output,stock_names,fund_names)
    df.at[idx,'prompt'] = prompts['common_prompt']
    df.at[idx,'prompt2'] = prompts['correct_prompt']

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

- 分类思维链推理

In [ ]:
df['glm4_output'] = ''

- 后处理

In [ ]:
df['final_output'] = df['glm4_output'].apply(lambda x: post_process(x))